New approach trying normalize with Google i18n address

if that doesn't work, I'll normalize, then parse them then try matching up

In [5]:
import numpy as np
import pandas as pd

from address_normalize.paths import DATA_DIR

raw_dir = DATA_DIR / 'raw'

files = list(raw_dir.glob('Master Project Address List*.xlsx'))
files.sort()

file_path = files[-1]

data = pd.read_excel(file_path, sheet_name=['E911 Data', 'Phase_I', 'Phase_II', 'Phase_III'])

df_e911 = data['E911 Data']
main_df = data['Phase_I']

In [6]:
main_df.head()

,OBJECTID,REF,ESITEID,Build Phase,ADDRESS,E911 ADDRESS,OWNER 1,OWNER 2,GRAND LIST ADDRESS,GRAND LIST = ADDRESS,...,COL_R,COL_S,COL_T,COL_U,COL_V,COL_W,COL_X,COL_Y,GNS Area,Construction Status
0,1547,1547,78274,Phase 1,114 S BINGHAM ST,114 S BINGHAM ST,114 S BINGHAM LLC,(QUINTTUS),15 ASHLAND AVE,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Included
1,1316,1316,139583,Phase 1,1896 OLD JERUSALEM RD,1896 OLD JERUSALEM RD,80 GOATS INC,,2001 OLD JERUSALEM RD,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,Included
2,189,189,184321,Phase 1,14 CHURCH ST,14 CHURCH ST,ABBOTT JACOB,ABBOTT KAREN LYNN,14 CHURCH ST,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Included
3,1382,1382,78171,Phase 1,24 PARK LN,24 PARK Ln,ABEL WILLA & SIMON,,24 PARK LN,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GoNetSpeed,Excluded
4,328,328,216370,Phase 1,156 MIDDLE RD,156 MIDDLE RD,ACCIAVIATTI BRUCE,SMITH COLLEEN,156 MIDDLE RD,Y,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,Included


In [7]:
df_e911['E911 ADDRESS']

0                32 BOOTH WOODS
1                        8 E St
2                  73 SCHOOL St
3                   206 MAIN St
4                   224 MAIN St
                  ...          
17430    574 ROBEROUTE YOUNG RD
17431                          
17432          151 VT R0UTE 116
17433         71 SPRING HILL Ln
17434               14 TATRO RD
Name: E911 ADDRESS, Length: 17435, dtype: object

In [8]:
sents = df_e911['E911 ADDRESS'].tolist()

In [9]:
df_e911.head()

,OBJECTID_1,E911 ADDRESS,OBJECTID,OWNER 1,OWNER 2,GRAND LIST ADDRESS,GRAND LIST CITY,GRAND LIST STATE,GRAND LIST ZIPCODE,SPAN,...,GLVAL_HS,GLVAL_NR,CRHOUSPCT,MUNGL1PCT,AOEGL_HS,AOEGL_NR,SHAPESTAre,SHAPESTLen,Shape_Length,Shape_Area
0,1,32 BOOTH WOODS,18621475,STAPLES WAYNE,MCENTEE SHEILA,32 BOOTH WOODS,VERGENNES,VT,05491,663-210-10001,...,1837,0,0,1837,1837,0,1231.173523,145.763504,145.763504,1231.173529
1,2,8 E St,18621476,LINCOLN GERONIMO LLC,,1341 ARNOLD BAY ROAD,PANTON,VT,05491,663-210-10002,...,0,1755,0,1755,0,1755,532.200386,92.414705,92.414705,532.200394
2,3,73 SCHOOL St,18621477,WHITNEY GRACE,,PO BOX 541,CROTON FALLS,NY,10519,663-210-10003,...,0,2213,0,2213,0,2213,880.616867,118.855189,118.855189,880.616861
3,4,206 MAIN St,18621478,SMALLEST CITY HOUSING LP,,PO BOX 156,VERGENNES,VT,05491,663-210-10004,...,0,2988,0,3284,0,2988,709.602440,109.360601,109.360601,709.602436
4,5,224 MAIN St,18621479,SMALLEST CITY HOUSING LP,,PO BOX 156,VERGENNES,VT,05491,663-210-10005,...,0,3314,0,3682,0,3314,1242.662354,167.708389,167.708389,1242.662357


In [2]:
from i18naddress import InvalidAddress, normalize_address

In [16]:
def norm_address(row):
    try:
        return normalize_address({
        'country_code': 'US',
        'country_area': 'Vermont',
        'city': row['GRAND LIST CITY'],
        'postal_code': row['GRAND LIST ZIPCODE'],
        'street_address': row['E911 ADDRESS']
        }
        )['street_address']
    except InvalidAddress:
        return False

In [21]:
df_e911['validated'] = df_e911.apply(norm_address, axis=1)

In [43]:
all(df_e911[df_e911.validated != df_e911['E911 ADDRESS']][['validated']])

True